In [9]:
import pandas as pd
pd.options.display.max_rows = 5
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import word_tokenize
import itertools
import pickle
import re

In [12]:
features = pickle.load( open( "seedwords.p", "rb" ) )

# Get WN info

In [13]:
for fk,fv in features.items():
    
    for sk,sv in fv['seeds'].items():
        
        wninfo=list()
        
        if isinstance(sv,set):
            
            for l in sv:
                
                wnl=wn.lemma_from_key(l)
                
                related={}
                
                nyms = ['antonyms','derivationally_related_forms','pertainyms']    
                for i in nyms:
                    words=list()
                    try:
                        if len(getattr(wnl, i)())!=0:
                            
                            #wninfo.append({'related':{i:set(getattr(wnl, i)())}})
                            for s in getattr(wnl, i)():
                                words.append(s.name())
                                #print(l, i, s)
                                #print(words)
                            related.update({i:words})       
                    except AttributeError as e: 
                        print(e)
                        pass

                nyms = ['hypernyms','instance_hypernyms','hyponyms','instance_hyponyms','member_holonyms','substance_holonyms','part_holonyms','member_meronyms','substance_meronyms','part_meronyms','topic_domains','region_domains','usage_domains','attributes','entailments','causes','also_sees','verb_groups','similar_tos']   
                for i in nyms:
                    words=list()
                    try:
                        if len(getattr(wnl.synset(), i)())!=0:
                            
                            #wninfo.append({'related':{i:set(getattr(wnl, i)())}})
                            for s in getattr(wnl.synset(), i)():
                                words.append(s.lemmas()[0].name())
                                #print(l, i, s)
                                #print(words)
                            related.update({i:words})   
                            #print(related)
                    except AttributeError as e: 
                        print(e)
                        pass
                    

                wninfo.append({l:{'word':wnl.name(),'pos':wnl.synset().pos(),'def':wnl.synset().definition(),'related':related}})
            features[fk]['seeds'][sk]=wninfo
            #print(features[fk]['seeds'][sk])

# Print and browse

In [19]:
class DictTable(dict):
    # Overridden dict class which takes a dict in the form {'a': 2, 'b': 3},
    # and renders an HTML Table in IPython Notebook.
    def _repr_html_(self):
        sorted_keys=sorted(self.keys(), key=lambda y: (self[y]['freq']),reverse=True)
        html = ["<table  style=' float: left'>"]
        html.append("<tr style='font-weight:bold'>")
        html.append("<td style='text-align: left !important'>{0}</td>".format('freq'))
        html.append("<td style='text-align: left !important'>{0}</td>".format('feature'))
        html.append("<td style='text-align: left !important'>{0}</td>".format('seed'))
        html.append("<td style='text-align: left !important'>{0}</td>".format('wnlemma'))
        html.append("<td style='text-align: left !important'>{0}</td>".format('pos'))
        html.append("<td style='text-align: left !important'>{0}</td>".format('def'))
        html.append("<td style='text-align: left !important'>{0}</td>".format('rel_source'))
        html.append("<td style='text-align: left !important'>{0}</td>".format('rel_list'))
        html.append("</tr>")
        # For each feature
        for i in sorted_keys:
            firstft=True
            fk=i
            fv=self[i]
            # For each seed
            for sk, sv in fv['seeds'].items():
                firstsd=True
                # If a set of lemmas, for each lemma, print a row
                if isinstance(sv,list):
                    for i in sv:
                        for lk,lv in i.items():
                            firstlm=True
                            if 'related' in lv.keys():
                                for rk,rv in lv['related'].items():
                                    html.append("<tr style='text-align: left !important'>")
                                    if firstft:
                                        html.append(str("<td style='text-align: left !important'>{0}</td>").format(fv['feature']))
                                        html.append(str("<td style='text-align: left !important'>{0}</td>").format(fv['freq']))
                                    elif firstft==False:
                                        html.append(str("<td style='text-align: left !important;color:#e1e1e1'>{0}</td>").format(fv['feature']))
                                        html.append(str("<td style='text-align: left !important;color:#e1e1e1'>{0}</td>").format(fv['freq']))
                                    if firstsd:
                                        html.append(str("<td style='text-align: left !important'>{0}</td>").format(sk))
                                    elif firstsd==False:
                                        html.append(str("<td style='text-align: left !important;color:#e1e1e1'>{0}</td>").format(sk))
                                    if firstlm:
                                        html.append("<td style='text-align: left !important'>{0}</td>".format(lv['word']))
                                        html.append("<td style='text-align: left !important'>{0}</td>".format(lv['pos']))
                                        html.append("<td style='text-align: left !important'>{0}</td>".format(lv['def']))
                                    elif firstlm==False:
                                        html.append(str("<td style='text-align: left !important;color:#e1e1e1'>{0}</td>").format(lv['word']))
                                        html.append(str("<td style='text-align: left !important;color:#e1e1e1'>{0}</td>").format(lv['pos']))
                                        html.append(str("<td style='text-align: left !important;color:#e1e1e1'>{0}</td>").format(lv['def']))
                                    html.append("<td style='text-align: left !important'>{0}</td>".format(rk))
                                    html.append("<td style='text-align: left !important'>{0}</td>".format(str(', '.join(rv))))
                                    html.append("</tr>")
                                    firstft=False
                                    firstsd=False
                                    firstlm=False

                            else:
                                print("")

                # If instance is not a set
                else: 
                        html.append("<tr style='table-layout: auto'>")
                        if firstft:
                            html.append(str("<td>{0}</td>").format(fv['feature']))
                            html.append(str("<td>{0}</td>").format(fv['freq']))
                        elif firstft==False:
                            html.append(str("<td style='color:#e1e1e1'>{0}</td>").format(fv['feature']))
                            html.append(str("<td style='color:#e1e1e1'>{0}</td>").format(fv['freq']))
                        html.append("<td>{0}</td>".format(sk))
                        html.append("<td>{0}</td>".format(sv))
                        html.append("<td>{0}</td>".format(''))
                        html.append("<td>{0}</td>".format(''))
                        html.append("<td>{0}</td>".format(''))
                        html.append("</tr>")
                        firstft=False

        html.append("</table>")
        return ''.join(html)

DictTable(features)

freq,feature,seed,wnlemma,pos,def,rel_source,rel_list
password,167,password,password,n,a secret word or phrase known only to a restricted group,hypernyms,"positive_identification, secret"
hacking,139,hacking,hacker,n,a programmer for whom computing is its own reward; may enjoy the challenge of breaking into other computers but does no harm,derivationally_related_forms,hack
hacking,139,hacking,hacker,n,a programmer for whom computing is its own reward; may enjoy the challenge of breaking into other computers but does no harm,hypernyms,programmer
hacking,139,hacking,hack,v,fix a computer program piecemeal until it works,derivationally_related_forms,hacker
hacking,139,hacking,hack,v,fix a computer program piecemeal until it works,hypernyms,program
safety,133,safety,safety,n,a safe place,hypernyms,area
safety,133,safety,safety,n,a safe place,hyponyms,harborage
safety,133,safety,safety,n,a device designed to prevent injury or accidents,hypernyms,device
safety,133,safety,safety,n,a device designed to prevent injury or accidents,hyponyms,"fender, safety_catch"
safety,133,safety,safety,n,the state of being certain that adverse effects will not be caused by some agent under defined conditions,antonyms,danger


In [7]:
alist=['child_abuse', 'child_neglect', 'cruelty', 'persecution']
', '.join(alist)

'child_abuse, child_neglect, cruelty, persecution'

In [8]:
class DictTable(dict):
    # Overridden dict class which takes a dict in the form {'a': 2, 'b': 3},
    # and renders an HTML Table in IPython Notebook.
    def _repr_html_(self):
        sorted_keys=sorted(self.keys(), key=lambda y: (self[y]['freq']),reverse=True)
        html = ["<table  style=' text-align: left'>"]
        html.append("<tr style='font-weight:bold'>")
        html.append("<td>{0}</td>".format('feature'))
        html.append("<td>{0}</td>".format('freq'))
        html.append("<td>{0}</td>".format('seed'))
        html.append("<td>{0}</td>".format('wnlemma'))
        html.append("<td>{0}</td>".format('pos'))
        html.append("<td>{0}</td>".format('def'))
        html.append("</tr>")
        # For each feature
        for i in sorted_keys:
            firstft=True
            ns=0
            nl=0
            fk=i
            fv=self[i]
            # For each seed
            for sk, sv in fv['seeds'].items():
                #print(sv)
                firstsd=True
                # If a set of lemmas, for each lemma, print a row
                if isinstance(sv,list):
                    for i in sv:
                        for lk,lv in i.items():
                            html.append("<tr style='text-align: left'>")
                            if firstft:
                                html.append(str("<td>{0}</td>").format(fv['feature']))
                                html.append(str("<td>{0}</td>").format(fv['freq']))
                            elif firstft==False:
                                html.append(str("<td style='color:#e1e1e1'>{0}</td>").format(fv['feature']))
                                html.append(str("<td style='color:#e1e1e1'>{0}</td>").format(fv['freq']))
                            if firstsd:
                                html.append(str("<td>{0}</td>").format(sk))
                            elif firstsd==False:
                                html.append(str("<td style='color:#e1e1e1'>{0}</td>").format(sk))
                            html.append("<td>{0}</td>".format(lv['word']))
                            html.append("<td>{0}</td>".format(lv['pos']))
                            html.append("<td>{0}</td>".format(lv['def']))
                            html.append("</tr>")
                            firstft=False
                            firstsd=False

                # If instance is not a set
                else: 
                        html.append("<tr style='table-layout: auto'>")
                        if firstft:
                            html.append(str("<td>{0}</td>").format(fv['feature']))
                            html.append(str("<td>{0}</td>").format(fv['freq']))
                        elif firstft==False:
                            html.append(str("<td style='color:#e1e1e1'>{0}</td>").format(fv['feature']))
                            html.append(str("<td style='color:#e1e1e1'>{0}</td>").format(fv['freq']))
                        html.append("<td>{0}</td>".format(sk))
                        html.append("<td>{0}</td>".format(sv))
                        html.append("<td>{0}</td>".format(''))
                        html.append("<td>{0}</td>".format(''))
                        html.append("<td>{0}</td>".format(''))
                        html.append("</tr>")
                        firstft=False

        html.append("</table>")
        return ''.join(html)

## Full list

In [ ]:
DictTable(features)

## Frequent features with no wn matches (>2)

In [ ]:
DictTable( {k:v for k,v in features.items() if v['freq'] > 2 and 'not in wn!' in v['seeds'].values()})

## Somewhat frequent (==2) not coded

In [ ]:
DictTable( {k:v for k,v in features.items() if v['freq'] == 2 and 'code!' in v['seeds'].values()})

In [ ]:
freqf={k:v for k,v in features.items() if v['freq'] > 2 and 'not in wn!' in v['seeds'].values()}

In [ ]:
freqf

In [ ]:
for fk,fv in freqf.items():
    for sk,sv in fv['seeds'].items():
        if isinstance(sv,list):
            for i in sv:
                for lk,lv in i.items():
                    print(lv)
        else:
            print(sv)